# Define Classes and create the SQLite database

> This module defines the classes we use to represent the PKM workflow.

In [ ]:
#| default_exp classdb

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from __future__ import annotations
import json
from enum import Enum
from typing import Union, ClassVar
from dataclasses import dataclass
from pydantic import BaseModel, field_serializer, field_validator, Field, computed_field
from fastlite import *
from fastcore.test import *
from hopsa import ossys

We use `from __future__ import annotations` to support forward references in type hints. To be precise in the `@classmethod` we create to keep track of all instances of the class.

## Using Pydantic with MiniDataAPI and SQLite

We want to use `Pydantic` Dataclasses to enable typechecking and validation. We also want to use the Dataclasses with the `MiniDataAPI` to create the tables in the `SQLite` database. But `SQLite` only has datatypes: `NULL`, `INTEGER`, `REAL`, `TEXT`, and `BLOB`. So no `list` or any of the Dataclass(Enum) types we use.

To be able to use both `Pydanctic` and the `MiniDataAPI` we will do two things:

1. Define a Pydantic Dataclass with the correct datatypes and a Dataclass that has the same fields as the Pydantic Dataclass, but with datatypes that can be used with SQLite.
2. We add `@field_serializer` methods to the Pydantic Dataclass that convert the fields to JSON strings when we use the method `.model_dump()` on the instance of the Pydantic Dataclass. These serialised JSON strings can then be added to the SQLite database.
3. We also use the `@field_validator` decorator to convert the JSON strings back to the correct datatypes when we load the data from the SQLite database back into the Pydantic Dataclass.

This way we can:

- create instances with the Pydantic Dataclass to have easy typechecking and validation.
- convert this instances to `MiniDataAPI` and `SQLite` friendly datatypes using `.model_dump()` on the instance, that we can then add to the database.
- load the data from the SQLite database back into the Pydantic Dataclass.

The exact implementation can be found below where the Classes are defined.

#### Track instances of classes

We also want to keep track of the instances available for each class. Therefore we need some higher order magic.

- a list in the class to store the instances
- a __init__ method to add the instance to the list
- a classmethod to get the list of instances

We can't just add a `_instances = []` statement to the Class, because Pydantic will then assume it is a model field (private attribute). We need to tell Pydantic to ignore the _instances class variable as a model field and treat is as a class variable. Therefore we need to import `ClassVar` from `typing` and use it to type the _instances variable.

## Enum Classes

First we define the possible values of the different variables that are available in the classes. We use the module `enum` to define **Enumerations**. We use this to bind the possible values to a variable name, making the code more readable and maintainable.

In [ ]:
#| export
class InformationType(Enum):
    """Information content types that flow through the PKM workflow."""
    BOOK = "book"
    RESEARCH_PAPER = "research_paper"
    DOCUMENT = "document"
    ANNOTATION = "annotations&highlights"
    NOTE = "note"
    EMAIL = "email"
    DISCORD_MESSAGE = "discord_message"
    WEB_ARTICLE = "web_article"
    YOUTUBE_VIDEO = "youtube_video"
    PODCAST = "podcast"
    PRODUCT_IDEA = "product_idea"
    PROJECT_IDEA = "project_idea"

class Method(Enum):
    """How actions are performed - manually or automatically."""
    NA = "na"
    MANUAL = "manual"
    AUTOMATIC = "automatic"

class Phase(Enum):
    """The five phases of the PKM workflow."""
    COLLECT = "collect"
    RETRIEVE = "retrieve"
    CONSUME = "consume"
    EXTRACT = "extract"
    REFINE = "refine"

class PhaseQuality(Enum):
    """Quality rating for how well a tool performs in each phase."""
    NA = "na"
    BAD = "bad"
    OK = "ok"
    GREAT = "great"

class OrganizationSystem(Enum):
    """How tools organize and structure information."""
    TAGS = "tags"
    FOLDERS = "folders"
    LINKS = "links"
    JOHNNY_DECIMAL = "johnny_decimal"

In [ ]:
Phase("refine")

<Phase.REFINE: 'refine'>

## Base class to use for database classes

In [ ]:
#| export
class SluggedModel(BaseModel):
    @computed_field
    @property
    def slug(self) -> str:
        return ossys.sanitize_name(self.name)

    @staticmethod
    def _fld(rec, name):
        return getattr(rec, name) if hasattr(rec, name) else rec[name]

## PKM Workflow Classes

Next we create a dataclass for each item we need to be present in the PKM workflow.

**Pydantic Dataclasses**

Used for typechecking.

When creating a new instance for an `InformationItem` the `toolflow` must be given as a list of `Tool` objects. The typechecking makes sure that any `Tool` object mentioned in the `toolfow` list, does exist as an actual `Tool` instance. So make sure to first create  all the `Tool` instances that are needed for an `InformationItem`, before creating the `InformationItem` instance.

:::{.callout-tip}
I had some serious trouble getting the Pydantic dataclass validations to work. One of the issues is described above and is about SQLite not supporting all datatypes. A second major issue is that the Pydantic Dataclasses reference each other. The `InformationItem` references the `Tool` in the `toolflow` field. I would also be convenient to store all the `InformationItem`s that can be used with a certain `Tool`, but in that case we would create a circular reference between `InformationItem` and `Tool`.

We decided to remove the `information_items` list from `Tool`. When we need to get all the `InformationItem`s that are supported by a `Tool` we can write a Python function or do a SQL-query on the SQLite database.

But then we are left with the fact that we want a list of `Tool`s that exist. These are the options considered:

- `toolflow: list[Tool]`
- `toolflow: list[Tool.name]`
- `toolflow: list[str]`

The last option is used in combination with validation to ensure each string is a valid Tool.name.

Here's why this is the best approach:

- Clean serialization (no complex object embedding)
- Human-readable in the database
- Type safety through validation
- Easy to query

The same goes for the `Improvement` class and the field `tool`.
:::

## Explainer for the Pydantic Dataclass `Tool`

### Method `Tool.from_db`

```python
@classmethod
def from_db(cls, db_record):
    phase_quality = PhaseQualityData(
        collect=PhaseQuality(db_record['collect_quality']),
        retrieve=PhaseQuality(db_record['retrieve_quality']),
        consume=PhaseQuality(db_record['consume_quality']),
        extract=PhaseQuality(db_record['extract_quality']),
        refine=PhaseQuality(db_record['refine_quality'])
    )
    org_systems = [OrganizationSystem(s) for s in json.loads(db_record['organization_system'])]
    return cls(
        id=db_record['id'],
        name=db_record['name'],
        description=db_record.get('description'),
        organization_system=org_systems,
        phase_quality=phase_quality,
        collect=db_record['collect'],
        retrieve=db_record['retrieve'],
        consume=db_record['consume'],
        extract=db_record['extract'],
        refine=db_record['refine']
    )
```

- **Input**: `db_record` is a row from the [Tools](cci:2://file:///home/jelle/code/infoflow/infoflow/classdb.py:110:8-126:31) table (e.g. a dict-like object from fastlite).
- **Rebuild `phase_quality`**:
  - DB stores each quality as a string (`'na' | 'bad' | 'ok' | 'great'`).
  - [PhaseQualityData](cci:2://file:///home/jelle/code/infoflow/infoflow/classdb.py:65:0-70:49) is reconstructed by wrapping each DB value with the [PhaseQuality](cci:2://file:///home/jelle/code/infoflow/infoflow/classdb.py:50:0-55:19) enum.
- **Rebuild `organization_system`**:
  - DB stores it as a JSON string representing a list of enum values, e.g. `["tags","folders"]`.
  - `json.loads` turns that into a Python list of strings.
  - Each string is turned into an [OrganizationSystem](cci:2://file:///home/jelle/code/infoflow/infoflow/classdb.py:57:0-62:37) enum instance.
- **Instantiate [Tool](cci:2://file:///home/jelle/code/infoflow/infoflow/classdb.py:72:0-133:316)**:
  - Pass all the primitive fields straight from `db_record`.
  - Pass the reconstructed `phase_quality` and `organization_system`.
- **Side-effect**:
  - [__init__](cci:1://file:///home/jelle/code/infoflow/infoflow/classdb.py:99:4-101:47) registers the instance in `Tool._instances` keyed by [slug](cci:1://file:///home/jelle/code/infoflow/infoflow/classdb.py:87:4-90:45).

**In short**: [from_db](cci:1://file:///home/jelle/code/infoflow/infoflow/classdb.py:129:4-133:316) converts DB‑friendly fields (JSON/string) back into enum-rich, structured Pydantic objects.

---

## 2. [InformationItem.from_db](cci:1://file:///home/jelle/code/infoflow/infoflow/classdb.py:225:4-235:117)

```python
@classmethod
def from_db(cls, db_record):
    toolflow = PhaseToolflowData(
        collect=cls._parse_toolflow(db_record['collect_toolflow']),
        retrieve=cls._parse_toolflow(db_record['retrieve_toolflow']),
        consume=cls._parse_toolflow(db_record['consume_toolflow']),
        extract=cls._parse_toolflow(db_record['extract_toolflow']),
        refine=cls._parse_toolflow(db_record['refine_toolflow'])
    )
    method = PhaseMethodData(
        collect=db_record['collect_method'],
        retrieve=db_record['retrieve_method'],
        consume=db_record['consume_method'],
        extract=db_record['extract_method'],
        refine=db_record['refine_method']
    )
    info_type = InformationType(db_record['info_type'])
    return cls(
        id=db_record['id'],
        name=db_record['name'],
        info_type=info_type,
        method=method,
        toolflow=toolflow
    )
```

The helper:

```python
@staticmethod
def _parse_toolflow(v):
    if v is None: return None
    if isinstance(v, str) and v.startswith('['): return json.loads(v)
    return v
```

- **Input**: a row from [InformationItems](cci:2://file:///home/jelle/code/infoflow/infoflow/classdb.py:196:8-211:32).
- **Rebuild `toolflow`**:
  - DB stores each `*_toolflow` as either:
    - `NULL`/`None`,
    - a plain string (single tool),
    - or a JSON-encoded list (multiple tools).
  - [_parse_toolflow](cci:1://file:///home/jelle/code/infoflow/infoflow/classdb.py:218:4-223:16):
    - returns `None` for `None`,
    - `json.loads` if it looks like a JSON list (starts with `'['`),
    - otherwise leaves the string as‑is.
  - Those values feed into [PhaseToolflowData](cci:2://file:///home/jelle/code/infoflow/infoflow/classdb.py:144:0-159:40) (which itself sanitizes names via validators).
- **Rebuild `method`**:
  - Note: DB stores method values as strings (`"manual"`, `"automatic"`, `"na"` or `None`).
  - [PhaseMethodData](cci:2://file:///home/jelle/code/infoflow/infoflow/classdb.py:136:0-141:47) fields are annotated as `Method | None`, so Pydantic will convert those strings to [Method](cci:2://file:///home/jelle/code/infoflow/infoflow/classdb.py:36:0-40:27) enums automatically.
- **Rebuild `info_type`**:
  - DB stores `info_type` as its string value (e.g. `"book"`).
  - This is wrapped into [InformationType](cci:2://file:///home/jelle/code/infoflow/infoflow/classdb.py:21:0-34:33).
- **Instantiate [InformationItem](cci:2://file:///home/jelle/code/infoflow/infoflow/classdb.py:163:0-235:117)**:
  - Pass the reconstructed `info_type`, `method`, `toolflow`, plus `id` and `name`.
  - [__init__](cci:1://file:///home/jelle/code/infoflow/infoflow/classdb.py:99:4-101:47) adds the instance to `InformationItem._instances`.

**In short**: [from_db](cci:1://file:///home/jelle/code/infoflow/infoflow/classdb.py:129:4-133:316) reverses the flattening that turned nested structures ([PhaseMethodData](cci:2://file:///home/jelle/code/infoflow/infoflow/classdb.py:136:0-141:47), [PhaseToolflowData](cci:2://file:///home/jelle/code/infoflow/infoflow/classdb.py:144:0-159:40), [InformationType](cci:2://file:///home/jelle/code/infoflow/infoflow/classdb.py:21:0-34:33)) into many DB columns + JSON strings.

---

## 3. [Improvement.from_db](cci:1://file:///home/jelle/code/infoflow/infoflow/classdb.py:295:4-298:171)

```python
@classmethod
def from_db(cls, db_record):
    phase = Phase(db_record['phase'])
    return cls(
        id=db_record['id'],
        title=db_record['title'],
        what=db_record['what'],
        why=db_record['why'],
        prio=db_record['prio'],
        tool=db_record['tool'],
        phase=phase
    )
```

- **Input**: a row from [Improvements](cci:2://file:///home/jelle/code/infoflow/infoflow/classdb.py:272:8-282:21).
- **Rebuild `phase`**:
  - DB stores `phase` as a string (e.g. `"collect"`).
  - Wrap in the [Phase](cci:2://file:///home/jelle/code/infoflow/infoflow/classdb.py:42:0-48:21) enum.
- **Instantiate [Improvement](cci:2://file:///home/jelle/code/infoflow/infoflow/classdb.py:238:0-298:171)**:
  - All other fields are passed through as-is.
  - [__init__](cci:1://file:///home/jelle/code/infoflow/infoflow/classdb.py:99:4-101:47) registers in `Improvement._instances`.
  - `tool` will be validated against [Tool.get_instances()](cci:1://file:///home/jelle/code/infoflow/infoflow/classdb.py:103:4-105:29) by the `field_validator`, so this will raise if the referenced tool slug is unknown.

**In short**: [from_db](cci:1://file:///home/jelle/code/infoflow/infoflow/classdb.py:129:4-133:316) here is simpler: it only converts the stored phase string to a [Phase](cci:2://file:///home/jelle/code/infoflow/infoflow/classdb.py:42:0-48:21) enum and lets Pydantic + validators do the rest.

---

## Conceptual pattern across all three

- **DB schema is “flattened”**:
  - Enums → strings.
  - Lists/tuples → JSON text.
  - Nested models → multiple columns.
- **[flatten_for_db](cci:1://file:///home/jelle/code/infoflow/infoflow/classdb.py:92:4-95:19)** does the forward conversion to DB‑friendly fields.
- **[from_db](cci:1://file:///home/jelle/code/infoflow/infoflow/classdb.py:129:4-133:316)** does the reverse: rehydrate Enums, nested Pydantic models, and collections from that DB representation, returning fully-typed domain models ready for use in the app.

If you want, I can also trace one concrete row (e.g. a [Tool](cci:2://file:///home/jelle/code/infoflow/infoflow/classdb.py:72:0-133:316) with multiple `organization_system` values) through [flatten_for_db](cci:1://file:///home/jelle/code/infoflow/infoflow/classdb.py:92:4-95:19) → SQLite row → [from_db](cci:1://file:///home/jelle/code/infoflow/infoflow/classdb.py:129:4-133:316) for a more step-by-step example.I'll open `infoflow/classdb.py`, find the `InformationItem`, `Tool`, and `Improvement` Pydantic dataclasses, and then walk through how their `from_db` methods behave, including any differences between them.

Here’s how [from_db](cci:1://file:///home/jelle/code/infoflow/infoflow/classdb.py:129:4-133:316) works for each of the three Pydantic models, based on [classdb.py](cci:7://file:///home/jelle/code/infoflow/infoflow/classdb.py:0:0-0:0).

---

## 1. [Tool.from_db](cci:1://file:///home/jelle/code/infoflow/infoflow/classdb.py:129:4-133:316)

```python
@classmethod
def from_db(cls, db_record):
    phase_quality = PhaseQualityData(
        collect=PhaseQuality(db_record['collect_quality']),
        retrieve=PhaseQuality(db_record['retrieve_quality']),
        consume=PhaseQuality(db_record['consume_quality']),
        extract=PhaseQuality(db_record['extract_quality']),
        refine=PhaseQuality(db_record['refine_quality'])
    )
    org_systems = [OrganizationSystem(s) for s in json.loads(db_record['organization_system'])]
    return cls(
        id=db_record['id'],
        name=db_record['name'],
        description=db_record.get('description'),
        organization_system=org_systems,
        phase_quality=phase_quality,
        collect=db_record['collect'],
        retrieve=db_record['retrieve'],
        consume=db_record['consume'],
        extract=db_record['extract'],
        refine=db_record['refine']
    )
```

- **Input**: `db_record` is a row from the [Tools](cci:2://file:///home/jelle/code/infoflow/infoflow/classdb.py:110:8-126:31) table (e.g. a dict-like object from fastlite).
- **Rebuild `phase_quality`**:
  - DB stores each quality as a string (`'na' | 'bad' | 'ok' | 'great'`).
  - [PhaseQualityData](cci:2://file:///home/jelle/code/infoflow/infoflow/classdb.py:65:0-70:49) is reconstructed by wrapping each DB value with the [PhaseQuality](cci:2://file:///home/jelle/code/infoflow/infoflow/classdb.py:50:0-55:19) enum.
- **Rebuild `organization_system`**:
  - DB stores it as a JSON string representing a list of enum values, e.g. `["tags","folders"]`.
  - `json.loads` turns that into a Python list of strings.
  - Each string is turned into an [OrganizationSystem](cci:2://file:///home/jelle/code/infoflow/infoflow/classdb.py:57:0-62:37) enum instance.
- **Instantiate [Tool](cci:2://file:///home/jelle/code/infoflow/infoflow/classdb.py:72:0-133:316)**:
  - Pass all the primitive fields straight from `db_record`.
  - Pass the reconstructed `phase_quality` and `organization_system`.
- **Side-effect**:
  - [__init__](cci:1://file:///home/jelle/code/infoflow/infoflow/classdb.py:99:4-101:47) registers the instance in `Tool._instances` keyed by [slug](cci:1://file:///home/jelle/code/infoflow/infoflow/classdb.py:87:4-90:45).

**In short**: [from_db](cci:1://file:///home/jelle/code/infoflow/infoflow/classdb.py:129:4-133:316) converts DB‑friendly fields (JSON/string) back into enum-rich, structured Pydantic objects.

---

## 2. [InformationItem.from_db](cci:1://file:///home/jelle/code/infoflow/infoflow/classdb.py:225:4-235:117)

```python
@classmethod
def from_db(cls, db_record):
    toolflow = PhaseToolflowData(
        collect=cls._parse_toolflow(db_record['collect_toolflow']),
        retrieve=cls._parse_toolflow(db_record['retrieve_toolflow']),
        consume=cls._parse_toolflow(db_record['consume_toolflow']),
        extract=cls._parse_toolflow(db_record['extract_toolflow']),
        refine=cls._parse_toolflow(db_record['refine_toolflow'])
    )
    method = PhaseMethodData(
        collect=db_record['collect_method'],
        retrieve=db_record['retrieve_method'],
        consume=db_record['consume_method'],
        extract=db_record['extract_method'],
        refine=db_record['refine_method']
    )
    info_type = InformationType(db_record['info_type'])
    return cls(
        id=db_record['id'],
        name=db_record['name'],
        info_type=info_type,
        method=method,
        toolflow=toolflow
    )
```

The helper:

```python
@staticmethod
def _parse_toolflow(v):
    if v is None: return None
    if isinstance(v, str) and v.startswith('['): return json.loads(v)
    return v
```

- **Input**: a row from [InformationItems](cci:2://file:///home/jelle/code/infoflow/infoflow/classdb.py:196:8-211:32).
- **Rebuild `toolflow`**:
  - DB stores each `*_toolflow` as either:
    - `NULL`/`None`,
    - a plain string (single tool),
    - or a JSON-encoded list (multiple tools).
  - [_parse_toolflow](cci:1://file:///home/jelle/code/infoflow/infoflow/classdb.py:218:4-223:16):
    - returns `None` for `None`,
    - `json.loads` if it looks like a JSON list (starts with `'['`),
    - otherwise leaves the string as‑is.
  - Those values feed into [PhaseToolflowData](cci:2://file:///home/jelle/code/infoflow/infoflow/classdb.py:144:0-159:40) (which itself sanitizes names via validators).
- **Rebuild `method`**:
  - Note: DB stores method values as strings (`"manual"`, `"automatic"`, `"na"` or `None`).
  - [PhaseMethodData](cci:2://file:///home/jelle/code/infoflow/infoflow/classdb.py:136:0-141:47) fields are annotated as `Method | None`, so Pydantic will convert those strings to [Method](cci:2://file:///home/jelle/code/infoflow/infoflow/classdb.py:36:0-40:27) enums automatically.
- **Rebuild `info_type`**:
  - DB stores `info_type` as its string value (e.g. `"book"`).
  - This is wrapped into [InformationType](cci:2://file:///home/jelle/code/infoflow/infoflow/classdb.py:21:0-34:33).
- **Instantiate [InformationItem](cci:2://file:///home/jelle/code/infoflow/infoflow/classdb.py:163:0-235:117)**:
  - Pass the reconstructed `info_type`, `method`, `toolflow`, plus `id` and `name`.
  - [__init__](cci:1://file:///home/jelle/code/infoflow/infoflow/classdb.py:99:4-101:47) adds the instance to `InformationItem._instances`.

**In short**: [from_db](cci:1://file:///home/jelle/code/infoflow/infoflow/classdb.py:129:4-133:316) reverses the flattening that turned nested structures ([PhaseMethodData](cci:2://file:///home/jelle/code/infoflow/infoflow/classdb.py:136:0-141:47), [PhaseToolflowData](cci:2://file:///home/jelle/code/infoflow/infoflow/classdb.py:144:0-159:40), [InformationType](cci:2://file:///home/jelle/code/infoflow/infoflow/classdb.py:21:0-34:33)) into many DB columns + JSON strings.

---

## 3. [Improvement.from_db](cci:1://file:///home/jelle/code/infoflow/infoflow/classdb.py:295:4-298:171)

```python
@classmethod
def from_db(cls, db_record):
    phase = Phase(db_record['phase'])
    return cls(
        id=db_record['id'],
        title=db_record['title'],
        what=db_record['what'],
        why=db_record['why'],
        prio=db_record['prio'],
        tool=db_record['tool'],
        phase=phase
    )
```

- **Input**: a row from [Improvements](cci:2://file:///home/jelle/code/infoflow/infoflow/classdb.py:272:8-282:21).
- **Rebuild `phase`**:
  - DB stores `phase` as a string (e.g. `"collect"`).
  - Wrap in the [Phase](cci:2://file:///home/jelle/code/infoflow/infoflow/classdb.py:42:0-48:21) enum.
- **Instantiate [Improvement](cci:2://file:///home/jelle/code/infoflow/infoflow/classdb.py:238:0-298:171)**:
  - All other fields are passed through as-is.
  - [__init__](cci:1://file:///home/jelle/code/infoflow/infoflow/classdb.py:99:4-101:47) registers in `Improvement._instances`.
  - `tool` will be validated against [Tool.get_instances()](cci:1://file:///home/jelle/code/infoflow/infoflow/classdb.py:103:4-105:29) by the `field_validator`, so this will raise if the referenced tool slug is unknown.

**In short**: [from_db](cci:1://file:///home/jelle/code/infoflow/infoflow/classdb.py:129:4-133:316) here is simpler: it only converts the stored phase string to a [Phase](cci:2://file:///home/jelle/code/infoflow/infoflow/classdb.py:42:0-48:21) enum and lets Pydantic + validators do the rest.

---

## Conceptual pattern across all three

- **DB schema is “flattened”**:
  - Enums → strings.
  - Lists/tuples → JSON text.
  - Nested models → multiple columns.
- **[flatten_for_db](cci:1://file:///home/jelle/code/infoflow/infoflow/classdb.py:92:4-95:19)** does the forward conversion to DB‑friendly fields.
- **[from_db](cci:1://file:///home/jelle/code/infoflow/infoflow/classdb.py:129:4-133:316)** does the reverse: rehydrate Enums, nested Pydantic models, and collections from that DB representation, returning fully-typed domain models ready for use in the app.

If you want, I can also trace one concrete row (e.g. a [Tool](cci:2://file:///home/jelle/code/infoflow/infoflow/classdb.py:72:0-133:316) with multiple `organization_system` values) through [flatten_for_db](cci:1://file:///home/jelle/code/infoflow/infoflow/classdb.py:92:4-95:19) → SQLite row → [from_db](cci:1://file:///home/jelle/code/infoflow/infoflow/classdb.py:129:4-133:316) for a more step-by-step example.

In [ ]:
#| export
class PhaseQualityData(BaseModel):
    collect: PhaseQuality = Field(PhaseQuality.NA)
    retrieve: PhaseQuality = Field(PhaseQuality.NA)
    consume: PhaseQuality = Field(PhaseQuality.NA)
    extract: PhaseQuality = Field(PhaseQuality.NA)
    refine: PhaseQuality = Field(PhaseQuality.NA)


In [ ]:
#| export
class Tool(SluggedModel):
    """Pydantic dataclass for tools. This Pydantic dataclass has several methods for converting to and from a 
    SQLite database. This is needed because SQLite doesn't support all used data types.
    !IMPORTANT: be sure to add or change the appropriate methods when adding or changing fields!"""
    id: int | None = Field(default=None, description="ID of the tool. This is automatically created when item is added to database.")
    name: str = Field(..., description="Name of the tool")
    description: str | None = Field(default=None, description="General intent/goal of the tool")
    organization_system: list[OrganizationSystem] = Field(..., description="Organization systems supported by the tool")
    phase_quality: PhaseQualityData = Field(..., description="Quality of the tool for each phase")
    collect: str | None = Field(default=None, description="Description how to use tool in collect phase")
    retrieve: str | None = Field(default=None, description="Description how to use tool in retrieve phase")
    consume: str | None = Field(default=None, description="Description how to use tool in consume phase")
    extract: str | None = Field(default=None, description="Description how to use tool in extract phase")
    refine: str | None = Field(default=None, description="Description how to use tool in refine phase")

    def __init__(self, **data):
        super().__init__(**data)
        type(self)._instances[self.slug] = self

    _instances: ClassVar[Dict[str, "Tool"]] = {}

    @classmethod
    def get_instances(cls) -> Dict[str, "Tool"]:
        return cls._instances

    def flatten_for_db(self):
        base = self.model_dump(exclude={'phase_quality', 'organization_system'})
        base.update(
            {'organization_system': json.dumps([org.value for org in self.organization_system]), 
            'collect_quality': self.phase_quality.collect.value, 
            'retrieve_quality': self.phase_quality.retrieve.value, 
            'consume_quality': self.phase_quality.consume.value, 
            'extract_quality': self.phase_quality.extract.value, 
            'refine_quality': self.phase_quality.refine.value}
            )
        return base

    @classmethod
    def get_db_schema(cls):
        """Returns a dataclass with SQLite-compatible field types."""
        @dataclass
        class Tools: # Class name is used as table name automatically
            id: int
            name: str
            description: str
            collect: str
            retrieve: str
            consume: str
            extract: str
            refine: str
            slug: str
            organization_system: str
            collect_quality: str
            retrieve_quality: str
            consume_quality: str
            extract_quality: str
            refine_quality: str
        return Tools
    
    @classmethod
    def from_db(cls, db_record):
        phase_quality = PhaseQualityData(
            collect=PhaseQuality(cls._fld(db_record, 'collect_quality')),
            retrieve=PhaseQuality(cls._fld(db_record, 'retrieve_quality')),
            consume=PhaseQuality(cls._fld(db_record, 'consume_quality')),
            extract=PhaseQuality(cls._fld(db_record, 'extract_quality')),
            refine=PhaseQuality(cls._fld(db_record, 'refine_quality')))
        org_systems = [OrganizationSystem(s) for s in json.loads(cls._fld(db_record, 'organization_system'))]
        return cls(id=cls._fld(db_record, 'id'), name=cls._fld(db_record, 'name'), description=cls._fld(db_record, 'description'), organization_system=org_systems, phase_quality=phase_quality, collect=cls._fld(db_record, 'collect'), retrieve=cls._fld(db_record, 'retrieve'), consume=cls._fld(db_record, 'consume'), extract=cls._fld(db_record, 'extract'), refine=cls._fld(db_record, 'refine'))

In [ ]:
#| export
class PhaseMethodData(BaseModel):
    collect: Method | None = Field(default=None)
    retrieve: Method | None = Field(default=None)
    consume: Method | None = Field(default=None)
    extract: Method | None = Field(default=None)
    refine: Method | None = Field(default=None)

Since `Pydantic` v2 Pydantic coerces sequences to tuples with a type of `tuple[str, ...]` Pydantic will accept a `list[str, ...]` as a valid input. Therefore we must include both `tuple` and `list` as a valid instance int the staticmethod `_san`. Else the validation will return `None` if a list is passed which is bad. It should return a `TypeError` to inform the user to use a tuple or it should just return the values as a tuple. The function `_san` now returns a tuple of strings if a list is passed.

In [ ]:
#| export
class PhaseToolflowData(BaseModel):
    collect: Union[str, tuple[str, ...], None] = Field(default=None)
    retrieve: Union[str, tuple[str, ...], None] = Field(default=None)
    consume: Union[str, tuple[str, ...], None] = Field(default=None)
    extract: Union[str, tuple[str, ...], None] = Field(default=None)
    refine: Union[str, tuple[str, ...], None] = Field(default=None)

    @staticmethod
    def _san(v):
        if v is None: return None
        if isinstance(v, str): return ossys.sanitize_name(v)
        if isinstance(v, (list, tuple)):
            return tuple([ossys.sanitize_name(i) for i in v])
    
    @field_validator('collect', 'retrieve', 'consume', 'extract', 'refine', mode='before')
    def _val(cls, v): return cls._san(v)


Example of creating an instance from `PhaseToolflowData` with all possible types of input: `str`, `list[str]`, `tuple[str]` and just skipping an entry to return `None`.

In [ ]:
PhaseToolflowData(collect=["Reader", "Recall"], retrieve="Recall", refine=("Obsidian", "Recall"))

PhaseToolflowData(collect=('reader', 'recall'), retrieve='recall', consume=None, extract=None, refine=('obsidian', 'recall'))

In [ ]:
#| export
class InformationItem(SluggedModel):
    """Pydantic dataclass for information items. This Pydantic dataclass has several methods for converting to and from a 
    SQLite database. This is needed because SQLite doesn't support all used data types.
    !IMPORTANT: be sure to add or change the appropriate methods when adding or changing fields!"""
    id: int | None = Field(default=None, description="ID of the information item. Automatically created when added to the database.")
    name: str = Field(..., description="Name of the information item")
    info_type: InformationType = Field(..., description="Type of information item, e.g. book, article, video, etc.")
    method: PhaseMethodData = Field(..., description="Methods used at each phase")
    toolflow: PhaseToolflowData = Field(..., description="Tools used for this item at each phase")

    def __init__(self, **data):
        super().__init__(**data)
        type(self)._instances[self.slug] = self

    @classmethod
    def get_instances(cls) -> Dict[str, "InformationItem"]:
        return cls._instances

    _instances: ClassVar[Dict[str, "InformationItem"]] = {}

    def flatten_for_db(self):
        base = self.model_dump(exclude={'method', 'toolflow'})
        base.update(
            {'collect_method': self.method.collect.value if self.method.collect else None, 
            'retrieve_method': self.method.retrieve.value if self.method.retrieve else None, 
            'consume_method': self.method.consume.value if self.method.consume else None, 
            'extract_method': self.method.extract.value if self.method.extract else None, 
            'refine_method': self.method.refine.value if self.method.refine else None,
            'collect_toolflow': json.dumps(self.toolflow.collect) if isinstance(self.toolflow.collect, (list, tuple)) else self.toolflow.collect,
            'retrieve_toolflow': json.dumps(self.toolflow.retrieve) if isinstance(self.toolflow.retrieve, (list, tuple)) else self.toolflow.retrieve,
            'consume_toolflow': json.dumps(self.toolflow.consume) if isinstance(self.toolflow.consume, (list, tuple)) else self.toolflow.consume,
            'extract_toolflow': json.dumps(self.toolflow.extract) if isinstance(self.toolflow.extract, (list, tuple)) else self.toolflow.extract,
            'refine_toolflow': json.dumps(self.toolflow.refine) if isinstance(self.toolflow.refine, (list, tuple)) else self.toolflow.refine}
            )
        return base

    @classmethod
    def get_instances(cls) -> Dict[str, InformationItem]:
        return cls._instances
    
    @classmethod
    def get_db_schema(cls):
        """Returns a dataclass with SQLite-compatible field types."""
        @dataclass
        class InformationItems: # Class name is used as table name automatically
            id: int
            name: str
            info_type: str
            slug: str
            collect_method: str
            retrieve_method: str
            consume_method: str
            extract_method: str
            refine_method: str
            collect_toolflow: str
            retrieve_toolflow: str
            consume_toolflow: str
            extract_toolflow: str
            refine_toolflow: str
        return InformationItems

    @field_serializer('info_type')
    def db_serialize(self, v):
        return v.value

    @staticmethod
    def _parse_toolflow(v):
        """function to parse toolflow. Need to handle none, string and list"""
        if v is None: return None
        if isinstance(v, str) and v.startswith('['): return json.loads(v)
        return v
        
    @classmethod
    def from_db(cls, db_record):
        # We need `json.loads` here because the toolflows are stored as JSON strings in the database to allow for lists
        toolflow = PhaseToolflowData(
            collect=cls._parse_toolflow(cls._fld(db_record, 'collect_toolflow')),
            retrieve=cls._parse_toolflow(cls._fld(db_record, 'retrieve_toolflow')),
            consume=cls._parse_toolflow(cls._fld(db_record, 'consume_toolflow')),
            extract=cls._parse_toolflow(cls._fld(db_record, 'extract_toolflow')),
            refine=cls._parse_toolflow(cls._fld(db_record, 'refine_toolflow')))
        method = PhaseMethodData(collect=cls._fld(db_record, 'collect_method'), retrieve=cls._fld(db_record, 'retrieve_method'), consume=cls._fld(db_record, 'consume_method'), extract=cls._fld(db_record, 'extract_method'), refine=cls._fld(db_record, 'refine_method'))
        info_type = InformationType(cls._fld(db_record, 'info_type'))
        return cls(id=cls._fld(db_record, 'id'), name=cls._fld(db_record, 'name'), info_type=info_type, method=method, toolflow=toolflow)

In [ ]:
#| export
class Improvement(SluggedModel):
    """Pydantic dataclass for improvements. This Pydantic dataclass has several methods for converting to and from a 
    SQLite database. This is needed because SQLite doesn't support all used data types.
    !IMPORTANT: be sure to add or change the appropriate methods when adding or changing fields!"""
    id: int | None = Field(default=None, description="ID of the improvement, is automatically created when inserted in db.")
    name: str = Field(..., description="Title of the improvement")
    what: str = Field(..., description="What needs to be improved")
    why: str = Field(..., description="Why is this improvement needed")
    how: str = Field(..., description="Some ideas how to build this improvement")
    prio: int = Field(..., description="Priority of the improvement. Lowest number is highest priority")
    tool: str = Field(..., description="slug of the Tool that needs improvement")
    phase: Phase = Field(..., description="Phase that needs improvement")

    def __init__(self, **data):
        super().__init__(**data)
        type(self)._instances[self.slug] = self

    @classmethod
    def get_instances(cls) -> Dict[str, "Improvement"]:
        return cls._instances

    _instances: ClassVar[Dict[str, "Improvement"]] = {}

    def flatten_for_db(self):
        return self.model_dump()

    @classmethod
    def get_db_schema(cls):
        """Returns a dataclass with SQLite-compatible field types."""
        @dataclass
        class Improvements: # Class name is used as table name automatically
            id: int
            name: str
            what: str
            why: str
            how: str
            prio: int
            tool: str
            phase: str
            slug: str
        return Improvements

    @field_serializer('phase')
    def db_serialize(self, v):
        return v.value
    
    @field_validator('tool')
    def validate_tool_names(cls, v):
        valid_tools = Tool.get_instances().keys()
        if v not in valid_tools: raise ValueError(f"Tool '{v}' does not exist")
        return v
    
    @classmethod
    def from_db(cls, db_record):
        phase = Phase(db_record['phase'])
        return cls(id=db_record['id'], name=db_record['name'], what=db_record['what'], why=db_record['why'], prio=db_record['prio'], tool=db_record['tool'], phase=phase)


Test creating instances

In [ ]:
def test_phase_quality_data():
    pqd = PhaseQualityData(collect=PhaseQuality.GREAT, retrieve=PhaseQuality.BAD, consume=PhaseQuality.OK, extract=PhaseQuality.NA, refine=PhaseQuality.GREAT)
    test_eq(pqd.collect, PhaseQuality.GREAT)
    test_eq(pqd.retrieve, PhaseQuality.BAD)

def test_tool_creation():
    tool = Tool(id=12, name="TestTool", organization_system=[OrganizationSystem.TAGS], phase_quality=PhaseQualityData(collect=PhaseQuality.GREAT, retrieve=PhaseQuality.BAD, consume=PhaseQuality.OK, extract=PhaseQuality.NA, refine=PhaseQuality.GREAT))
    test_eq(tool.name, "TestTool")
    test_eq(tool.slug, "testtool")
    test_eq(tool.phase_quality.collect, PhaseQuality.GREAT)
    return tool

def test_tool_flatten():
    tool = Tool(id=12, name="TestTool", organization_system=[OrganizationSystem.TAGS, OrganizationSystem.FOLDERS], phase_quality=PhaseQualityData(collect=PhaseQuality.GREAT, retrieve=PhaseQuality.BAD, consume=PhaseQuality.OK, extract=PhaseQuality.NA, refine=PhaseQuality.GREAT))
    flat = tool.flatten_for_db()
    test_eq(flat['collect_quality'], 'great')
    test_eq(flat['retrieve_quality'], 'bad')
    test_eq(flat['name'], 'TestTool')

def test_information_item():
    methods = PhaseMethodData(collect=Method.MANUAL, retrieve=None, consume=None, extract=None, refine=None)
    tools = PhaseToolflowData(collect="Reader", retrieve=("Recall", "Reader"), consume=None, extract=None, refine=None)
    item = InformationItem(id=16, name="Test Article", info_type=InformationType.WEB_ARTICLE, method=methods, toolflow=tools)
    test_eq(item.method.collect, Method.MANUAL)
    test_eq(item.toolflow.collect, "reader")
    test_eq(item.toolflow.retrieve, ("recall", "reader"))
    return item

def test_information_item_flatten():
    methods = PhaseMethodData(collect=Method.MANUAL, retrieve=None, consume=None, extract=None, refine=None)
    tools = PhaseToolflowData(collect=("Reader", "Recall"), retrieve="Recall", consume=None, extract=None, refine=None)
    item = InformationItem(id=16,name="Test Article", info_type=InformationType.WEB_ARTICLE, method=methods, toolflow=tools)
    flat = item.flatten_for_db()
    test_eq(flat['collect_method'], 'manual')
    test_eq(flat['retrieve_method'], None)
    test_eq(flat['collect_toolflow'], '["reader", "recall"]')
    test_eq(flat['retrieve_toolflow'], 'recall')

def test_improvement():
    imp = Improvement(id=18, name="Fix Search", what="Better search in Reader", why="Current search is bad", how="By thinking for yoursel", prio=1, tool="testtool", phase=Phase.RETRIEVE)
    test_eq(imp.name, "Fix Search")
    test_eq(imp.phase, Phase.RETRIEVE)
    test_eq(imp.flatten_for_db()['phase'], 'retrieve')
    return imp

In [ ]:
OrganizationSystem("tags")

<OrganizationSystem.TAGS: 'tags'>

In [ ]:
OrganizationSystem.FOLDERS

<OrganizationSystem.FOLDERS: 'folders'>

In [ ]:
test_phase_quality_data()
test_tool_creation()
test_tool_flatten()
test_information_item()
test_information_item_flatten()
test_improvement()

Improvement(id=18, name='Fix Search', what='Better search in Reader', why='Current search is bad', how='By thinking for yoursel', prio=1, tool='testtool', phase=<Phase.RETRIEVE: 'retrieve'>, slug='fix_search')

## SQLite database

#### Create and connect

Connect to the database in the `main.py`. We should also enable foreign key constraints. These are disabled by default in Sqlite.

For testing purposes in this module we will use `db = database(":memory:")` to create an in-memory database.

In [ ]:
#| export
def create_db(
    loc: str = "../data/infoflow.db" # Location of the SQLite database
) -> Database:
    db = database(loc)
    db.execute("PRAGMA foreign_keys = ON;")
    return db

::: {.callout-tip}
We can add foreign key constraints to the tables using the `transform` method from `sqlite_utils`.

```python
inf_tbl.transform(add_foreign_keys=[("<field_name>", "<table_name_to_connect>", "<field_name_in_table_to_connect>")])
```
:::

But for now we won't use foreign key constraints.

We can create the tables using the function `create_tables_from_pydantic` but we won't be using it. The tables will be created automatically when we insert the first item for each table.

In [ ]:
#| export
def create_tables_from_pydantic(
    db: Database,
    classes: List[BaseModel]) -> Tuple[Table, Table, Table]:

    for c in classes:
        db.create(c.get_db_schema(), transform=True)

Tests and usage examples

In [ ]:
db = create_db(":memory:")

In [ ]:
create_tables_from_pydantic(db, [Tool, InformationItem, Improvement])

In [ ]:
db.t

improvements, information_items, tools

In [ ]:
db.t.improvements.c

how, id, name, phase, prio, slug, tool, what, why

Testing of manipulating database by adding a "something" field to the Improvement dataclass.

In [ ]:
Improvement.get_instances()

{'fix_search': Improvement(id=18, name='Fix Search', what='Better search in Reader', why='Current search is bad', how='By thinking for yoursel', prio=1, tool='testtool', phase=<Phase.RETRIEVE: 'retrieve'>, slug='fix_search')}

In [ ]:
db.t.improvements.insert(test_improvement().flatten_for_db())

{'id': 18,
 'name': 'Fix Search',
 'what': 'Better search in Reader',
 'why': 'Current search is bad',
 'how': 'By thinking for yoursel',
 'prio': 1,
 'tool': 'testtool',
 'phase': 'retrieve',
 'slug': 'fix_search'}

In [ ]:
Improvement.get_instances()

{'fix_search': Improvement(id=18, name='Fix Search', what='Better search in Reader', why='Current search is bad', how='By thinking for yoursel', prio=1, tool='testtool', phase=<Phase.RETRIEVE: 'retrieve'>, slug='fix_search')}

In [ ]:
before_cols = [c.name for c in L(db.t.improvements.c)]

In [ ]:
class Improvement(SluggedModel):
    """Pydantic dataclass for improvements. This Pydantic dataclass has several methods for converting to and from a 
    SQLite database. This is needed because SQLite doesn't support all used data types.
    !IMPORTANT: be sure to add or change the appropriate methods when adding or changing fields!"""
    id: int | None = Field(default=None, description="ID of the improvement, is automatically created when inserted in db.")
    name: str = Field(..., description="Title of the improvement")
    what: str = Field(..., description="What needs to be improved")
    why: str = Field(..., description="Why is this improvement needed")
    how: str = Field(..., description="Some ideas how to build this improvement")
    prio: int = Field(..., description="Priority of the improvement. Lowest number is highest priority")
    tool: str = Field(..., description="slug of the Tool that needs improvement")
    something: str = Field(..., description="Something")
    phase: Phase = Field(..., description="Phase that needs improvement")

    def __init__(self, **data):
        super().__init__(**data)
        type(self)._instances[self.slug] = self

    @classmethod
    def get_instances(cls) -> Dict[str, "Improvement"]:
        return cls._instances

    _instances: ClassVar[Dict[str, "Improvement"]] = {}

    def flatten_for_db(self):
        return self.model_dump()

    @classmethod
    def get_db_schema(cls):
        """Returns a dataclass with SQLite-compatible field types."""
        @dataclass
        class Improvements: # Class name is used as table name automatically
            id: int
            name: str
            what: str
            why: str
            how: str
            prio: int
            tool: str
            something: str
            phase: str
            slug: str
        return Improvements

    @field_serializer('phase')
    def db_serialize(self, v):
        return v.value
    
    @field_validator('tool')
    def validate_tool_names(cls, v):
        valid_tools = Tool.get_instances().keys()
        if v not in valid_tools: raise ValueError(f"Tool '{v}' does not exist")
        return v
    
    @classmethod
    def from_db(cls, db_record):
        phase = Phase(db_record['phase'])
        return cls(id=db_record['id'], name=db_record['name'], what=db_record['what'], why=db_record['why'], prio=db_record['prio'], tool=db_record['tool'], phase=phase)


In [ ]:
def test_improvement_newschema():
    imp = Improvement(id=20, name="Fix Search again", what="Better search in Reader", why="Current search is bad", how="By thinking for yoursel", prio=1, tool="testtool", something="omnious", phase=Phase.RETRIEVE)
    test_eq(imp.name, "Fix Search again")
    test_eq(imp.phase, Phase.RETRIEVE)
    test_eq(imp.flatten_for_db()['phase'], 'retrieve')
    return imp
test_improvement_newschema()
Improvement.get_instances()

{'fix_search_again': Improvement(id=20, name='Fix Search again', what='Better search in Reader', why='Current search is bad', how='By thinking for yoursel', prio=1, tool='testtool', something='omnious', phase=<Phase.RETRIEVE: 'retrieve'>, slug='fix_search_again')}

In [ ]:
create_tables_from_pydantic(db, [Improvement])

In [ ]:
after_cols = [c.name for c in L(db.t.improvements.c)]

In [ ]:
if before_cols == after_cols:
    raise ValueError("Improvements table conversion does not work correctly.")

Check if data is still in database

In [ ]:
db.t.improvements['18'].name

'Fix Search'

In [ ]:
test_eq(db.t.improvements['18'].name, "Fix Search")

#### Add data to the database

Add the created instances to the SQLite tables

In [ ]:
Tool.get_instances()

{'testtool': Tool(id=12, name='TestTool', description=None, organization_system=[<OrganizationSystem.TAGS: 'tags'>, <OrganizationSystem.FOLDERS: 'folders'>], phase_quality=PhaseQualityData(collect=<PhaseQuality.GREAT: 'great'>, retrieve=<PhaseQuality.BAD: 'bad'>, consume=<PhaseQuality.OK: 'ok'>, extract=<PhaseQuality.NA: 'na'>, refine=<PhaseQuality.GREAT: 'great'>), collect=None, retrieve=None, consume=None, extract=None, refine=None, slug='testtool')}

In [ ]:
InformationItem.get_instances()

{'test_article': InformationItem(id=16, name='Test Article', info_type=<InformationType.WEB_ARTICLE: 'web_article'>, method=PhaseMethodData(collect=<Method.MANUAL: 'manual'>, retrieve=None, consume=None, extract=None, refine=None), toolflow=PhaseToolflowData(collect=('reader', 'recall'), retrieve='recall', consume=None, extract=None, refine=None), slug='test_article')}

In [ ]:
Improvement.get_instances()

{'fix_search_again': Improvement(id=20, name='Fix Search again', what='Better search in Reader', why='Current search is bad', how='By thinking for yoursel', prio=1, tool='testtool', something='omnious', phase=<Phase.RETRIEVE: 'retrieve'>, slug='fix_search_again')}

Add a single instance to the SQLite table

In [ ]:
tst = {'name': 'NeoReader', 'collect': None, 'retrieve': None, 'consume': None, 'extract': None, 'refine': None, 'slug': 'neoreader', 'organization_system': '["folders"]', 'collect_quality': 'ok', 'retrieve_quality': 'bad', 'consume_quality': 'great', 'extract_quality': 'na', 'refine_quality': 'na'}

In [ ]:
db.t.tools.insert(tst)

{'id': 1,
 'name': 'NeoReader',
 'description': None,
 'collect': None,
 'retrieve': None,
 'consume': None,
 'extract': None,
 'refine': None,
 'slug': 'neoreader',
 'organization_system': '["folders"]',
 'collect_quality': 'ok',
 'retrieve_quality': 'bad',
 'consume_quality': 'great',
 'extract_quality': 'na',
 'refine_quality': 'na'}

In [ ]:
tst2 = {'id': 1, 'name': 'NeoReader', 'collect': "hiers staat nu wat", 'retrieve': None, 'consume': None, 'extract': None, 'refine': None, 'slug': 'neoreader', 'organization_system': '["folders"]', 'collect_quality': 'ok', 'retrieve_quality': 'bad', 'consume_quality': 'great', 'extract_quality': 'na', 'refine_quality': 'na'}

In [ ]:
db.t.tools.update(tst2)

{'id': 1,
 'name': 'NeoReader',
 'description': None,
 'collect': 'hiers staat nu wat',
 'retrieve': None,
 'consume': None,
 'extract': None,
 'refine': None,
 'slug': 'neoreader',
 'organization_system': '["folders"]',
 'collect_quality': 'ok',
 'retrieve_quality': 'bad',
 'consume_quality': 'great',
 'extract_quality': 'na',
 'refine_quality': 'na'}

In [ ]:
Improvement.get_instances()['fix_search_again'].flatten_for_db()

{'id': 20,
 'name': 'Fix Search again',
 'what': 'Better search in Reader',
 'why': 'Current search is bad',
 'how': 'By thinking for yoursel',
 'prio': 1,
 'tool': 'testtool',
 'something': 'omnious',
 'phase': 'retrieve',
 'slug': 'fix_search_again'}

In [ ]:
db.t.improvements.c

how, id, name, phase, prio, slug, something, tool, what, why

In [ ]:
db.t.improvements.insert(Improvement.get_instances()['fix_search_again'].flatten_for_db())
db.t.information_items.insert(InformationItem.get_instances()['test_article'].flatten_for_db())


{'id': 16,
 'name': 'Test Article',
 'info_type': 'web_article',
 'slug': 'test_article',
 'collect_method': 'manual',
 'retrieve_method': None,
 'consume_method': None,
 'extract_method': None,
 'refine_method': None,
 'collect_toolflow': '["reader", "recall"]',
 'retrieve_toolflow': 'recall',
 'consume_toolflow': None,
 'extract_toolflow': None,
 'refine_toolflow': None}

In [ ]:
[r for r in db.t.information_items.rows_where()]

[{'id': 16,
  'name': 'Test Article',
  'info_type': 'web_article',
  'slug': 'test_article',
  'collect_method': 'manual',
  'retrieve_method': None,
  'consume_method': None,
  'extract_method': None,
  'refine_method': None,
  'collect_toolflow': '["reader", "recall"]',
  'retrieve_toolflow': 'recall',
  'consume_toolflow': None,
  'extract_toolflow': None,
  'refine_toolflow': None}]

Add multiple instances to the SQLite table

In [ ]:
db.t.tools.insert_all([t.flatten_for_db() for t in Tool.get_instances().values()])

<Table tools (id, name, description, collect, retrieve, consume, extract, refine, slug, organization_system, collect_quality, retrieve_quality, consume_quality, extract_quality, refine_quality)>

In [ ]:
[t.flatten_for_db() for t in Tool.get_instances().values()]

[{'id': 12,
  'name': 'TestTool',
  'description': None,
  'collect': None,
  'retrieve': None,
  'consume': None,
  'extract': None,
  'refine': None,
  'slug': 'testtool',
  'organization_system': '["tags", "folders"]',
  'collect_quality': 'great',
  'retrieve_quality': 'bad',
  'consume_quality': 'ok',
  'extract_quality': 'na',
  'refine_quality': 'great'}]

In [ ]:
[r for r in db.t.tools.rows]

[{'id': 1,
  'name': 'NeoReader',
  'description': None,
  'collect': 'hiers staat nu wat',
  'retrieve': None,
  'consume': None,
  'extract': None,
  'refine': None,
  'slug': 'neoreader',
  'organization_system': '["folders"]',
  'collect_quality': 'ok',
  'retrieve_quality': 'bad',
  'consume_quality': 'great',
  'extract_quality': 'na',
  'refine_quality': 'na'},
 {'id': 12,
  'name': 'TestTool',
  'description': None,
  'collect': None,
  'retrieve': None,
  'consume': None,
  'extract': None,
  'refine': None,
  'slug': 'testtool',
  'organization_system': '["tags", "folders"]',
  'collect_quality': 'great',
  'retrieve_quality': 'bad',
  'consume_quality': 'ok',
  'extract_quality': 'na',
  'refine_quality': 'great'}]

#### Retrieve data from the database

Now retrieve the info from the database as intances from the Pydantic Dataclass using the `MiniDataAPI` from answer.ai.

In [ ]:
type(db.t.tools)

apswutils.db.Table

In [ ]:
db.t.tools()

[{'id': 1,
  'name': 'NeoReader',
  'description': None,
  'collect': 'hiers staat nu wat',
  'retrieve': None,
  'consume': None,
  'extract': None,
  'refine': None,
  'slug': 'neoreader',
  'organization_system': '["folders"]',
  'collect_quality': 'ok',
  'retrieve_quality': 'bad',
  'consume_quality': 'great',
  'extract_quality': 'na',
  'refine_quality': 'na'},
 {'id': 12,
  'name': 'TestTool',
  'description': None,
  'collect': None,
  'retrieve': None,
  'consume': None,
  'extract': None,
  'refine': None,
  'slug': 'testtool',
  'organization_system': '["tags", "folders"]',
  'collect_quality': 'great',
  'retrieve_quality': 'bad',
  'consume_quality': 'ok',
  'extract_quality': 'na',
  'refine_quality': 'great'}]

In [ ]:
tool = Tool.from_db(db.t.tools()[0])
tool

Tool(id=1, name='NeoReader', description=None, organization_system=[<OrganizationSystem.FOLDERS: 'folders'>], phase_quality=PhaseQualityData(collect=<PhaseQuality.OK: 'ok'>, retrieve=<PhaseQuality.BAD: 'bad'>, consume=<PhaseQuality.GREAT: 'great'>, extract=<PhaseQuality.NA: 'na'>, refine=<PhaseQuality.NA: 'na'>), collect='hiers staat nu wat', retrieve=None, consume=None, extract=None, refine=None, slug='neoreader')

In [ ]:
db.t.tools()

[{'id': 1,
  'name': 'NeoReader',
  'description': None,
  'collect': 'hiers staat nu wat',
  'retrieve': None,
  'consume': None,
  'extract': None,
  'refine': None,
  'slug': 'neoreader',
  'organization_system': '["folders"]',
  'collect_quality': 'ok',
  'retrieve_quality': 'bad',
  'consume_quality': 'great',
  'extract_quality': 'na',
  'refine_quality': 'na'},
 {'id': 12,
  'name': 'TestTool',
  'description': None,
  'collect': None,
  'retrieve': None,
  'consume': None,
  'extract': None,
  'refine': None,
  'slug': 'testtool',
  'organization_system': '["tags", "folders"]',
  'collect_quality': 'great',
  'retrieve_quality': 'bad',
  'consume_quality': 'ok',
  'extract_quality': 'na',
  'refine_quality': 'great'}]

In [ ]:
#| export
def dict_from_db(
        db_table: Table,
        class_table: BaseModel
    ) -> dict[str, BaseModel]:
    """Converts a database table to a dictionary of pydantic models."""
    d = {}
    for t in db_table():
        slug = getattr(t, "slug") if hasattr(t, "slug") else t["slug"]
        d[slug] = class_table.from_db(t)
    return d

In [ ]:
tools_dict = dict_from_db(db.t.tools, Tool)
item_dict = dict_from_db(db.t.information_items, InformationItem)

In [ ]:
tools_dict

{'neoreader': Tool(id=1, name='NeoReader', description=None, organization_system=[<OrganizationSystem.FOLDERS: 'folders'>], phase_quality=PhaseQualityData(collect=<PhaseQuality.OK: 'ok'>, retrieve=<PhaseQuality.BAD: 'bad'>, consume=<PhaseQuality.GREAT: 'great'>, extract=<PhaseQuality.NA: 'na'>, refine=<PhaseQuality.NA: 'na'>), collect='hiers staat nu wat', retrieve=None, consume=None, extract=None, refine=None, slug='neoreader'),
 'testtool': Tool(id=12, name='TestTool', description=None, organization_system=[<OrganizationSystem.TAGS: 'tags'>, <OrganizationSystem.FOLDERS: 'folders'>], phase_quality=PhaseQualityData(collect=<PhaseQuality.GREAT: 'great'>, retrieve=<PhaseQuality.BAD: 'bad'>, consume=<PhaseQuality.OK: 'ok'>, extract=<PhaseQuality.NA: 'na'>, refine=<PhaseQuality.GREAT: 'great'>), collect=None, retrieve=None, consume=None, extract=None, refine=None, slug='testtool')}

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()